# 기본 설정

In [ ]:
import path

In [ ]:
# !pip -q install transformers peft accelerate safetensors bitsandbytes datasets tqdm huggingface-hub

In [ ]:
import os, json, torch
from typing import List, Dict, Optional,Any
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer, TextIteratorStreamer, TextGenerationPipeline
from peft import PeftModel
import pandas as pd

HF_CACHE = "hf_cache"
BASE_MODEL = "../model/ko-gemma-2-9b-it"

In [ ]:
egen = pd.read_csv("../dataset/DPO/egen_test.csv")
teto = pd.read_csv("../dataset/DPO/teto_test.csv")

In [ ]:
# device = 0 if torch.cuda.is_available() else "cpu"
# device

### 모델 목록

In [ ]:
# 1) 모델 구성표
# 'base'는 어댑터 없이 바로 사용
# sft/dpo 결과는 어댑터 경로만 바꿔서 사용

MODEL_TABLE: Dict[str, Dict[str, Optional[str]]] = {
    # 1) Base (어댑터 없음, 4bit 로딩)
    "base": {"base_model": BASE_MODEL,"adapter": None,},

    # 2) SFT → DPO (최종 DPO 결과 어댑터)
    #   폴더: dpo_output_egen / dpo_output_teto
    "sft_dpo-egen": {"base_model": BASE_MODEL, "adapter": "../model/dpo_output_egen"},
    "sft_dpo-teto": {"base_model": BASE_MODEL, "adapter": "../model/dpo_output_teto"},

    # 3) DPO-only 라인: SFT(on DPO-ds) → DPO (최종)
    #   폴더: dpo_only_output_egen / dpo_only_output_teto
    "dpoonly_sft_dpo-egen": {"base_model": BASE_MODEL, "adapter": "../model/dpo_only_output_egen"},
    "dpoonly_sft_dpo-teto": {"base_model": BASE_MODEL, "adapter": "../model/dpo_only_output_teto"},
}

import os
for k, v in MODEL_TABLE.items():
    base_ok = os.path.exists(v["base_model"])
    ad_ok = (v["adapter"] is None) or os.path.exists(v["adapter"])
    print(f"{k:24} base={'OK' if base_ok else 'MISSING'}  adapter={'OK' if ad_ok else 'MISSING'} -> {v}")

In [ ]:
# 2) 프롬프트 빌더 (에겐/테토)
def build_prompt(user_text: str, persona: str = None) -> str:
    if persona == "egen":
        sys = (
            "당신은 **다정**하고, **감성**적이며, 섬세한 **에스트로겐 스타일**의 사람입니다."
            "MBTI는 INFP입니다. 말투는 **부드럽고 공감하는 어조**입니다. 음식 취향은 건강식, 취미는 독서와 산책을 좋아합니다."
            "감정 표현을 잘하며, 옷스타일은 따뜻한 색상의 내추럴 스타일입니다. 질문에 대해 친절하고 따뜻하게, **여성적으로 대답**하세요."
            "친한 친구와 대화하듯 반말로 편하게 얘기하고 간결하게 답하며 느낌표나 물음표를 적극 활용하세요."
        )
    else:
        sys = (
            "당신은 **자신감**있고, **직설적**이며, **논리적**인 **테스토스테론 스타일**의 사람입니다."
            "MBTI는 ESTP입니다. 말투는 **간결하고 단호한 어조**입니다. 음식 취향은 고기류를 선호하고, 취미는 운동과 게임을 좋아합니다."
            "감정보다 논리를 중시하며, 옷스타일은 깔끔하고 세련된 슈트 스타일입니다. 질문에 대해 논리적이고 직설적으로, **남성적으로 대답**하세요."
            "친한 친구와 대화하듯 반말로 편하게 얘기하고 간결하게 답하세요."
        )

    # 간단한 system/user 포맷(모델에 맞게 조절 가능)
    return f"<|system|>\n{sys}\n<|end|>\n{user_text}\n<|assistant|>\n"


In [ ]:
def get_terminators(tokenizer) -> List[int]:
    ids = [tokenizer.eos_token_id]
    try:
        extra = tokenizer.convert_tokens_to_ids("<|end|>")
        if isinstance(extra, int) and extra not in (tokenizer.eos_token_id, tokenizer.unk_token_id):
            ids.append(extra)
    except Exception:
        pass
    return ids

In [ ]:
adapters = {
    "egen": [None,
             "../model/dpo_output_egen",
             "../model/dpo_only_output_egen"],
    "teto": [None,
             "../model/dpo_output_teto",
             "../model/dpo_only_output_teto"],
}

In [ ]:
import multiprocessing as mp
import os
import torch
import gc
from functools import partial

GEN_KW = dict(
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.05,
)

def inference_single_gpu(args):
    """단일 GPU에서 추론하는 함수"""
    data_chunk, persona, adapter_path, gpu_id = args
    
    # 특정 GPU 설정
    os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
    torch.cuda.set_device(0)
    
    print(f"🚀 Starting GPU {gpu_id} with {len(data_chunk)} samples")
    
    model_label = "base" if adapter_path is None else os.path.basename(adapter_path.rstrip("/"))
    
    # 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, cache_dir=HF_CACHE)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        cache_dir=HF_CACHE,
    )
    
    if adapter_path:
        model = PeftModel.from_pretrained(model, adapter_path, is_trainable=False)
    
    model.eval()
    
    # 배치 추론
    results = []
    batch_size = 16
    
    for i in range(0, len(data_chunk), batch_size):
        batch = data_chunk[i:i+batch_size]
        prompts = [build_prompt(str(text), persona=persona) for text in batch]
        
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        
        with torch.inference_mode():
            outputs = model.generate(**inputs, **GEN_KW, pad_token_id=tokenizer.eos_token_id)
        
        for j, output in enumerate(outputs):
            input_len = inputs['input_ids'][j].shape[0]
            response = tokenizer.decode(output[input_len:], skip_special_tokens=True)
            
            results.append({
                "model": model_label,
                "persona": persona,
                "prompt": batch[j],
                "output": response.strip(),
                "gpu_id": gpu_id
            })
    
    # 메모리 정리
    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    
    print(f"✅ GPU {gpu_id} completed {len(results)} inferences")
    return results

def infer_multi_gpu_notebook(df, text_col, persona, num_gpus=5):
    """노트북에서 멀티 GPU 추론"""
    adapter_paths = adapters[persona]
    all_results = []
    
    for adapter_path in adapter_paths:
        print(f"🔄 Processing adapter: {adapter_path}")
        
        # 데이터를 GPU 개수만큼 분할
        data = df[text_col].values
        chunk_size = len(data) // num_gpus
        data_chunks = [data[i*chunk_size:(i+1)*chunk_size] for i in range(num_gpus)]
        
        # 마지막 청크에 남은 데이터 추가
        if len(data) % num_gpus != 0:
            data_chunks[-1] = data[(num_gpus-1)*chunk_size:]
        
        # 각 GPU에 할당할 작업 준비
        tasks = []
        for gpu_id, chunk in enumerate(data_chunks):
            if len(chunk) > 0:
                tasks.append((chunk.tolist(), persona, adapter_path, gpu_id))
        
        # 멀티프로세싱으로 병렬 실행
        print(f"🚀 Starting {len(tasks)} GPU processes...")
        
        with mp.Pool(processes=len(tasks)) as pool:
            chunk_results = pool.map(inference_single_gpu, tasks)
        
        # 결과 병합
        for chunk_result in chunk_results:
            all_results.extend(chunk_result)
    
    return all_results


In [ ]:
text_col = "prompt"
num_gpus = 5  # 사용 가능한 GPU 개수

# 에겐 데이터 추론 시작
all_results = infer_multi_gpu_notebook(egen, text_col, "egen", num_gpus=num_gpus)

# 결과 저장
results_df = pd.DataFrame(all_results)
results_df.to_csv("output/egen_test.csv", index=False)

In [ ]:
# 테토 데이터 추론 시작
all_results = infer_multi_gpu_notebook(teto, text_col, "teto", num_gpus=num_gpus)

# 결과 저장
results_df = pd.DataFrame(all_results)
results_df.to_csv("output/teto_test.csv", index=False)

In [ ]:
# Human Evaluation 용 질문 생성
data = ["나 오늘 우울해서 빵 샀어.",
    "헬스장 가기 너무 귀찮아.",
    "친구가 뭔가 나를 싫어하는 거 같아.",
    "인간관계에서 지치는 순간은 언제야?",
    "요즘 좀 무기력하고 아무것도 하기 싫어."
]

df = pd.DataFrame(data, columns = ['prompt'])

In [ ]:
egen = infer_multi_gpu_notebook(df, 'prompt', "egen", num_gpus=num_gpus)
teto = infer_multi_gpu_notebook(df, 'prompt', "teto", num_gpus=num_gpus)

In [ ]:
egen = pd.DataFrame(egen)
teto = pd.DataFrame(teto)

df = pd.concat([egen, teto], ignore_index=True)

mask = df["model"].eq("base")
df.loc[mask, "model"] = "base_" + df.loc[mask, "persona"].astype(str)

In [ ]:
# 1) 6개 타깃 모델 이름(순서 고정)
ORDER = [
    "base-egen", "dpo_output_egen", "dpo_only_output_egen",
    "base-teto", "dpo_output_teto", "dpo_only_output_teto",
]

wide_df = df.pivot_table(
    index='prompt',     # 새로운 데이터프레임의 행(index)이 될 열
    columns='model',    # 새로운 데이터프레임의 열(column)이 될 열
    values='output',    # 표의 셀을 채울 값
    aggfunc='first'     # 중복값이 있을 경우 첫번째 값을 사용 (일반적으로 각 prompt-model 쌍은 유일하므로 'first' 사용)
)

In [ ]:
df_for_records = wide_df.reset_index()

# to_json 함수를 사용하여 변환
json_list_format = df_for_records.to_json(
    orient='records',    # 각 행을 json 객체로 변환하여 리스트에 담는 방식
    indent=4,            # json 결과물을 예쁘게 들여쓰기
    force_ascii=False    # 한글이 깨지지 않도록 아스키 변환을 비활성화
)

with open('output/output_data.json', 'w', encoding='utf-8') as file:
    file.write(json_list_format)